In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# cd drive/MyDrive/SMAI_Project

Mounted at /content/drive


## **IMPORTING LIBRARIES**


In [3]:
import numpy as np
import nltk
import nltk.data
import spacy
import os
import tensorflow_hub as hub
import copy
!pip install rouge
import spacy 
from tqdm import tqdm
from rouge import Rouge

nltk.download('punkt')

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


2023-04-19 23:11:08.731428: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-19 23:11:08.732687: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 23:11:08.758935: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 23:11:08.759532: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 23:11:09.378613: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

[nltk_data] Downloading package punkt to /home/siddhant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
src_file = "./summarizer/DocsData.txt"
tar_file= "./summarizer/target_new.txt"
k=200

## **READ DOCUMENTS FROM FILE**

In [8]:
# read source file into a list of list:
def read_file(path, file_name, read_lead_only=False, read_first_doc=False):
    f = open(os.path.join(path, file_name),"r")
    lines = f.readlines()
    src_list = []
    tag="story_separator_special_tag"
    for line in lines:
        # remove tag; uncomment below for baseline
        line = line.replace(tag, "")
        # tokenzie line to sentences
        sent_list = sent_detector.tokenize(line.strip())
        src_list.append(sent_list)
    return src_list



src_list = read_file('', src_file)[:100]
with open(tar_file, 'r') as file:
    target = file.read().replace('\n', '')

[["GOP Eyes Gains As Voters In 11 States Pick Governors Enlarge this image toggle caption Jim Cole / AP Jim Cole / AP Voters in 11 states will pick their governors tonight , and Republicans appear on track to increase their numbers by at least one , with the potential to extend their hold to more than two - thirds of the nation 's top state offices .", 'Eight of the gubernatorial seats up for grabs are now held by Democrats ; three are in Republican hands .', "Republicans currently hold 29 governorships , Democrats have 20 , and Rhode Island 's Gov.", 'Lincoln Chafee is an Independent .', "Polls and race analysts suggest that only three of tonight 's contests are considered competitive , all in states where incumbent Democratic governors are n't running again : Montana , New Hampshire and Washington .", 'While those state races remain too close to call , Republicans are expected to wrest the North Carolina governorship from Democratic control , and to easily win GOP - held seats in Uta

## **Function to get USE embeddings of the sentences**

In [ ]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence]) 
  embedding = embedding.numpy()
  return embedding[0]

## **Functions to compute distance between each pair of sentences**

In [ ]:
def cosine_dist(A1,B1):
  A = np.array(A1)
  B = np.array(B1)
  mag_A = np.linalg.norm(A)
  mag_B = np.linalg.norm(B)

  dot_AB = np.dot(A,B)

  return (dot_AB)/(mag_A*mag_B)


In [ ]:
def compute_cosine_mat(em_li, thres):
  length = len(em_li)
  cos_mat = np.zeros([length, length])
  conn_list = []
  for i in range(length):

    for j in range(i+1,length):

      cos_mat[i][j] = cosine_dist(em_li[index_map[i]],em_li[index_map[j]])
      cos_mat[j][i] = cos_mat[i][j]
      if(cos_mat[i][j]>thres):
        # print(i," --> ",j)
        conn_list.append((index_map[i],index_map[j]))
  return cos_mat, conn_list


In [ ]:
def dfs(cur):
  if(flag[cur]==1):
    return
  # print(cur)
  flag[cur] = 1
  for key in graph_con[cur]:
    if(flag[key]==0):
      dfs(key)
  return

In [ ]:
# src_file = "Documents.txt"
# src_list = read_file('', src_file)[:100]

# with open(tar_file, 'r') as file:
#     target = file.read().replace('\n', '')

## **Based on Cosine Similarity assigning egdes between similar sentences**

In [4]:
emb_doc = {}
conn_list_All = []
N_nodes = 0
sen_list=[]
ki=0
ct=0
index_map = {}     #  1  -->  1_9  
inv_index_map = {}  #  1_9 -->  1
for doc in src_list:
  s_li = []
  kj=0
  for sen in doc:
    emb_doc[str(ki)+"_"+str(kj)] = get_use_embedding(sen)

    index_map[ct] = str(ki)+"_"+str(kj)
    inv_index_map[str(ki)+"_"+str(kj)] = ct
    sen_list.append(sen)
    kj+=1
    ct+=1
    N_nodes+=1
  ki+=1

NameError: name 'src_list' is not defined

In [ ]:
Ae, c_li = compute_cosine_mat(emb_doc,0.05)
graph_con = {}
flag = {}
for key in inv_index_map:
  graph_con[key] = []
  flag[key] = 0
for tup in c_li:
  graph_con[tup[0]].append(tup[1])
  graph_con[tup[1]].append(tup[0])

## **Obtaining the Laplacian Matrix**

In [ ]:
Are = copy.deepcopy(Ae)
for i in range(len(Are)):
  for j in range(len(Are[i])):
    if(i==j):
      Are[i][j] = len(graph_con[index_map[i]])
    elif(Are[i][j]>=0.05):
      Are[i][j]=-1
    else:
      Are[i][j]=0
# sa=[]
# for i in range(0,197):
#   sa.append(len(graph_con[index_map[i]]))

## **Calculating Eigen values & vector**

In [ ]:
Are = Are.astype('int64')
w, v = np.linalg.eig(Are)
v = np.real_if_close(v, tol=1)
w= np.real_if_close(w, tol=1)
eigen_asc = []

for i in range(len(w)):
  eigen_asc.append((w[i],i))
eigen_asc.sort(reverse=True)

w=np.sort(w)

# for i in w:
#   print(i)


In [ ]:

li_k_vec = []
ind_eigv = [] 

for i in range(k):
  ind_eigv.append(eigen_asc[i][1])
  li_k_vec.append(v[eigen_asc[i][1]])

N_coor = np.array(li_k_vec).T

## **Applying K-means Clustering on Embedding obtained above**

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=k, n_init=10, random_state=0)
kmeans.fit(N_coor)

cluster_dict={}
for i in range(k):
  cluster_dict[i]=[]

for i in range(len(sen_list)):
  k_num=kmeans.labels_[i]
  cluster_dict[k_num].append(sen_list[i])


In [ ]:
# zr=np.zeros(100)
# for i in kmeans.labels_:
#   zr[i]=zr[i]+1
  
# for i in zr:
#   print(i)


## **POS tagging Senetences to feed it in Compressor**

In [ ]:
import spacy 
spacy_=spacy.load("en_core_web_sm")

def pos_tag(sentences):
  out=[]
  for sentence in sentences:
    sentence =sentence.replace("/","")
    word_dict=spacy_(sentence)
    temp=[]
    for word in word_dict:
      tagg=word.text+"/"+word.tag_
      temp.append(tagg)
    out.append(' '.join(temp))
  return out


## **Compressing each cluster**

In [ ]:
import takahe
def compress_cluster(sentences, nb_words):
    compresser = takahe.word_graph(sentences, nb_words = nb_words,lang = 'en',punct_tag = "." )
    candidates = compresser.get_compression(10)
    reranker = takahe.keyphrase_reranker(sentences,candidates,lang='en')
    reranked_candidates = reranker.rerank_nbest_compressions()
    if len(reranked_candidates)>0:
        _, path = reranked_candidates[0]
        result = ' '.join([u[0] for u in path])
    else:
        result=' '
    return result

In [ ]:
summary=[]
for i in tqdm(range(k)):
  summary.append(compress_cluster(pos_tag(cluster_dict[i]),5))
ans="".join(summary)

100%|██████████| 200/200 [13:01<00:00,  3.91s/it]


## **Comparing obtained summary with target summary**

In [ ]:
from rouge import Rouge
rouge = Rouge()
rouge.get_scores(ans,target)

[{'rouge-1': {'f': 0.38677861775750105,
   'p': 0.6643772893772893,
   'r': 0.2727956382778718},
  'rouge-2': {'f': 0.11452645722286277,
   'p': 0.2662402558464921,
   'r': 0.07295432139336182},
  'rouge-l': {'f': 0.3545248525968986,
   'p': 0.6089743589743589,
   'r': 0.25004700131603685}}]